In [1]:
import requests 
import json
from lxml import html
import time

from pprint import pprint

import pandas as pd

from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

pd.set_option('display.max_rows', None)

# Задание

Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

Сложить собранные новости в БД

Минимум один сайт, максимум - все три

In [2]:
# Наименование новости: //div/a/h2[contains(@class,'mg-card__title')]
# Ссылка на новость: //div/a[contains(@class,'mg-card__link')]
# Дата публикации: //div/span[contains(@class,'mg-card-source__time')]

In [3]:
# Mongo DB, создание коллекции

client = MongoClient('127.0.0.1', 27017)
db = client['news_database']
news_coll = db.news_yandex
news_coll

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news_database'), 'news_yandex')

In [23]:
# Функция добавляет Новости в созданную коллекцию
def insert_news(collection, add_article):
    try:
        collection.insert_one(add_article)
        print(f"Добавление новости {add_article['name']}")
    except dke:
        print(f"Новость по ссылке {add_article['_id']} уже существует в базе")

In [24]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}
#header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}

response = requests.get('https://yandex.ru/news?', headers=header)

dom = html.fromstring(response.text)

#items = dom.xpath("//div[contains(@class,'mg-grid__col mg-grid__col_xs_12 mg-grid__col_sm_9')]")
items = dom.xpath("//article[contains(@class,'mg-card')]")

news = []

for item in items:
    article = {}

    name = item.xpath(".//h2[contains(@class,'mg-card__title')]/text()")
    link = item.xpath(".//a[contains(@class,'mg-card__link')]/@href")
    date = item.xpath(".//span[contains(@class,'mg-card-source__time')]/text()")
    
    # Приведение array в string для более читабильного вида в базе Mongo DB
    article['_id'] = ' '.join(link)
    article['name'] = ' '.join(name)
    article['link'] = ' '.join(link)
    article['date'] = ' '.join(date)

    news.append(article)
    
    # Добавления новости в базу
    insert_news(news_coll, article)

#pprint(news)

Новость по ссылке https://yandex.ru/news/story/Telo_rossijskogo_diplomata_nashli_vozle_posolstva_vBerline--e148dedef8d7bc113a5361b46110dbf0?lang=ru&rubric=index&fan=1&stid=noO5PUqfPI4MRt1YREE_&t=1636118947&tt=true&persistent_id=168436492 уже существует в базе
Новость по ссылке https://yandex.ru/news/story/Vice-premer_Golikova_zayavila_ob_ukhudshenii_situacii_sCOVID-19_vdesyati_regionakh_Rossii--64b21c81e9e6e47948610b37c46894ab?lang=ru&rubric=index&fan=1&stid=BA6VAUa4Z-XcwwROkGTn&t=1636118947&tt=true&persistent_id=168432252 уже существует в базе
Новость по ссылке https://yandex.ru/news/story/Genprokuratura_odobrila_reshenie_Suda_Niderlandov_ob_otmene_Rossiej_vyplaty_podelu_YUKOSa--3b79078d30f85edad1d1a045edbb8236?lang=ru&rubric=index&fan=1&stid=tgftXtAE8vi5PyMzIiBb&t=1636118947&tt=true&persistent_id=168393492 уже существует в базе
Новость по ссылке https://yandex.ru/news/story/Eshhe_12_rabotnikov_skoroj_Evrejskoj_avtonomnoj_oblasti_uvolilis_posle_otkaza_otvakcinacii--7f1b864454b0d6d3675

In [26]:
df = pd.DataFrame(news)

In [27]:
df

,_id,name,link,date
0,https://yandex.ru/news/story/Telo_rossijskogo_...,Тело российского дипломата нашли возле посольс...,https://yandex.ru/news/story/Telo_rossijskogo_...,19:26
1,https://yandex.ru/news/story/Vice-premer_Golik...,Вице-премьер Голикова заявила об ухудшении сит...,https://yandex.ru/news/story/Vice-premer_Golik...,19:23
2,https://yandex.ru/news/story/Genprokuratura_od...,Генпрокуратура одобрила решение Суда Нидерланд...,https://yandex.ru/news/story/Genprokuratura_od...,19:24
3,https://yandex.ru/news/story/Eshhe_12_rabotnik...,Еще 12 работников скорой Еврейской автономной ...,https://yandex.ru/news/story/Eshhe_12_rabotnik...,19:19
4,https://yandex.ru/news/story/Delo_onapadenii_n...,Дело о нападении на мужчину с ребенком в Новой...,https://yandex.ru/news/story/Delo_onapadenii_n...,19:19
5,https://yandex.ru/news/story/VMoskve_vdvoe_uve...,В Москве вдвое увеличены выплаты донорам плазм...,https://yandex.ru/news/story/VMoskve_vdvoe_uve...,18:40
6,https://yandex.ru/news/story/Mehr_Moskvy_Sobya...,Мэр Москвы Собянин заявил о снижении числа еже...,https://yandex.ru/news/story/Mehr_Moskvy_Sobya...,17:27
7,https://yandex.ru/news/story/Petr_Biryukov_Zav...,Петр Бирюков: Завершен ремонт Лужнецкой эстака...,https://yandex.ru/news/story/Petr_Biryukov_Zav...,17:30
8,https://yandex.ru/news/story/Nachalos_stroitel...,Началось строительство нового участка трассы «...,https://yandex.ru/news/story/Nachalos_stroitel...,19:18
9,https://yandex.ru/news/story/Sozdateli_avtorsk...,Создатели авторской анимации в Москве смогут п...,https://yandex.ru/news/story/Sozdateli_avtorsk...,19:07
